In [9]:
!rm -rf ./experiments
!rm -rf ./some/other/path
%config InlineBackend.figure_format = 'retina'

# Basic Usage

<div align="center">
    <img src="logo.svg" style="width: min(100%, 450px)">
</div>

`digital-experiments` works straight out of the box:

In [1]:
from digital_experiments import experiment

@experiment
def square(x):
    return x * x

[square(i) for i in range(5)]

[0, 1, 4, 9, 16]

If you have `pandas` installed, you can also use the [to_dataframe](core-api.rst#digital_experiments.core.Experiment.to_dataframe) method:

In [2]:
square.to_dataframe()

,id,result,config.x
0,2023-12-14_15-21-10_082597,0,0
1,2023-12-14_15-21-10_138612,1,1
2,2023-12-14_15-21-10_187359,4,2
3,2023-12-14_15-21-10_221718,9,3
4,2023-12-14_15-21-10_255920,16,4


In [3]:
square.observations()

[Observation(2023-12-14_15-21-10_082597, {'x': 0} → 0),
 Observation(2023-12-14_15-21-10_138612, {'x': 1} → 1),
 Observation(2023-12-14_15-21-10_187359, {'x': 2} → 4),
 Observation(2023-12-14_15-21-10_221718, {'x': 3} → 9),
 Observation(2023-12-14_15-21-10_255920, {'x': 4} → 16)]

## Observations

Each [Observation](core-api.rst#digital_experiments.core.Observation) object is a light-weight wrapper around:

- a unique identifier (implemented as a timestamped string)
- the exact configuration (args, kwargs and defaults) used to run the experiment
- the result of the experiment (the return value of the function)
- a dictionary of metadata that internal and user-defined [callback hooks](callbacks-api.rst) can use to store other relevant information

In [4]:
import json

_dict = square.observations()[0]._asdict()
print(json.dumps(_dict, indent=4))

{
    "id": "2023-12-14_15-21-10_082597",
    "config": {
        "x": 0
    },
    "result": 0,
    "metadata": {
        "git": {
            "branch": "git-metadata",
            "commit": "9a5d7f3ab3f7082143642f9a2ee6a11466a9c418",
            "remote": "https://github.com/jla-gardner/digital-experiments.git"
        },
        "timing": {
            "total": {
                "start": "2023-12-14 15:21:10",
                "end": "2023-12-14 15:21:10",
                "duration": 0.054958
            }
        },
        "code": "@experiment\ndef square(x):\n    return x * x\n"
    }
}


By default, `digital-experiments` times how long the experiment took, and the exact code that was run. The latter is particularly useful when we're rapdily iterating on an experiment's code, and want to be able to reproduce the results of a previous run. 

Extra timing information can be added to this metadata by using the [time_block](core-api.rst#digital_experiments.time_block) context.

## Backends

By default, `digital-experiments` stores each observation in its own `.pkl` file located at `./experiments/<experiment_name>/<id>.pkl`:

In [5]:
!ls ./experiments/square

2023-12-14_15-21-10_082597.pkl 2023-12-14_15-21-10_221718.pkl
2023-12-14_15-21-10_138612.pkl 2023-12-14_15-21-10_255920.pkl
2023-12-14_15-21-10_187359.pkl


Other backends are available (see the [complete list here](backends-api.rst)), or you can [implement your own](backends-api.rst).

You can also specify the root directory for a given experiment by passing the `root` argument to the [@experiment](core-api.rst#digital_experiments.experiment) decorator, or by setting the `DE_ROOT` environment variable:

In [6]:
from pathlib import Path

@experiment(root=Path("some/other/path"))
def cube(x):
    return x ** 3

cube(4)
!ls ./some/other/path

2023-12-14_15-21-10_724272.pkl


## Artefacts

`digital-experiments` assigns and provides a unique directory on disk per run of an experiment. This can be accessed within an experiment using the [current_dir](core-api.rst#digital_experiments.current_dir) function. Any files saved to this directory during the experiment are available _post hoc_ via the [artefacts](core-api.rst#digital_experiments.core.Experiment.artefacts) function.

In [7]:
from digital_experiments import current_dir

@experiment
def saving_experiment():
    (current_dir() / 'test.txt').write_text('hello world')

saving_experiment()
id = saving_experiment.observations()[0].id
saving_experiment.artefacts(id)

[PosixPath('experiments/saving_experiment/storage/2023-12-14_15-21-10_892895/test.txt')]